In [20]:
import cbpro
import pandas as pd

from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass
from coinbase.wallet.client import Client

In [21]:
coinbase_API_key = Coinbase_API
coinbase_API_secret = Coinbase_secret
client = Client(coinbase_API_key, coinbase_API_secret)

total = 0
message = []

accounts = client.get_accounts()
for wallet in accounts.data:
    message.append( str(wallet['name']) + ' ' + str(wallet['native_balance']) )
    value = str( wallet['native_balance']).replace('USD','')
    total += float(value)
message.append( 'Total Balance: ' + 'USD ' + str(total) )
print ('\n'.join( message ))

AuthenticationError: APIError(id=authentication_error): invalid api key

In [22]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):  
        self.url = 'wss://ws-feed.exchange.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count =+ 1
        msg_type = msg.get('type', None)
        if msg_type == 'ticker':
            time_val = msg.get('time',('-'*27))
            price_val = msg.get('price', None)
            price_val = float(price_val) if price_val is not None else 'None'
            product_id = msg.get('product_id', None)
            
            print(f'{time_val:30} {price_val:3f} {product_id}\tchannel type:{msg_type}')
    
    def on_close(self):
        print(f'<---Websocket connection closed--->\n\tTotal messages: {self.message_count}')

In [23]:
stream = TextWebsocketClient(products=['ETH-USD'],channels=['ticker'])
stream.open()

AttributeError: 'TextWebsocketClient' object has no attribute 'open'

### Authenticated Client

In [32]:
from api import Coinbase_API
from api import Coinbase_secret
from api import Coinbase_pass

api_key = Coinbase_API
api_secret = Coinbase_secret
api_pass = Coinbase_pass

In [33]:
url = 'https://api.pro.coinbase.com'
client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass,
    api_url = url)

In [34]:
accounts = client.get_accounts()

In [49]:
accounts

[{'id': 'e645fa11-abb3-4b61-b564-019baa0899d2',
  'currency': '1INCH',
  'balance': '0.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '0',
  'profile_id': 'ca3ccf79-3a0b-4da7-9ec6-481c3ef7c75f',
  'trading_enabled': True},
 {'id': 'b8d8dbcb-c9c9-4b9e-977a-18fe46f641d1',
  'currency': 'AAVE',
  'balance': '0.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '0',
  'profile_id': 'ca3ccf79-3a0b-4da7-9ec6-481c3ef7c75f',
  'trading_enabled': True},
 {'id': 'a4c0b5ee-8666-4c86-b19d-3d49bb9b1d18',
  'currency': 'ACH',
  'balance': '0.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '0',
  'profile_id': 'ca3ccf79-3a0b-4da7-9ec6-481c3ef7c75f',
  'trading_enabled': True},
 {'id': '664ee358-f01f-4d44-a96c-82291d2396d7',
  'currency': 'ADA',
  'balance': '0.0099790000000000',
  'hold': '0.0000000000000000',
  'available': '0.009979',
  'profile_id': 'ca3ccf79-3a0b-4da7-9ec6-481c3ef7c75f',
  'trading_enabled': True},
 {'id': '41e143bf-0e32-4ca2-91

In [54]:
for acc in accounts:
    currency = acc.get('currency')
    if currency == 'SHIB':
        acc_id = acc.get('id')

In [55]:
acc_history = client.get_account_history(acc_id)

In [56]:
import json

for hist in acc_history:
    print(json.dumps(hist, indent=1))

{
 "id": "17636714172",
 "amount": "180466498.0000000000000000",
 "balance": "318167743.0000000000000000",
 "created_at": "2022-03-01T14:37:08.79916Z",
 "type": "match",
 "details": {
  "order_id": "1809c539-810b-4dd1-a85d-d2dd3e3afd50",
  "product_id": "SHIB-USD",
  "trade_id": "67678090"
 }
}
{
 "id": "17636714169",
 "amount": "3125000.0000000000000000",
 "balance": "137701245.0000000000000000",
 "created_at": "2022-03-01T14:37:08.793874Z",
 "type": "match",
 "details": {
  "order_id": "1809c539-810b-4dd1-a85d-d2dd3e3afd50",
  "product_id": "SHIB-USD",
  "trade_id": "67678089"
 }
}
{
 "id": "17636714166",
 "amount": "4903522.0000000000000000",
 "balance": "134576245.0000000000000000",
 "created_at": "2022-03-01T14:37:08.788577Z",
 "type": "match",
 "details": {
  "order_id": "1809c539-810b-4dd1-a85d-d2dd3e3afd50",
  "product_id": "SHIB-USD",
  "trade_id": "67678088"
 }
}
{
 "id": "17636714163",
 "amount": "59148907.0000000000000000",
 "balance": "129672723.0000000000000000",
 "create